In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import gzip
import json
import ast
import os

# ETL Process:

User_Reviews

In [2]:
# Define the path to the raw user reviews data file
file_path = os.path.join("datasets", "raw", "user_reviews.json.gz")

# Initialize an empty list to store reviews
reviews_row = []

# Open the compressed JSON file in read-text mode with UTF-8 encoding
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    # Iterate over each line in the file
    for line in file.readlines():
        # Use ast.literal_eval to convert each line from JSON string to Python dict
        reviews_row.append(ast.literal_eval(line))

# Create a DataFrame from the list of dictionaries
df_reviews = pd.DataFrame(reviews_row)

In [3]:
# Expanding 'reviews'
reviews_expanded = df_reviews.explode('reviews')

# Creating a DataFrame
reviews_df = reviews_expanded['reviews'].apply(pd.Series)

# Concatenating 'user_id' with the reviews DataFrame:
df_reviews = pd.concat([reviews_expanded['user_id'], reviews_df], axis=1).reset_index(drop=True)

In [4]:
df_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [5]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      59333 non-null  object 
 1   funny        59305 non-null  object 
 2   posted       59305 non-null  object 
 3   last_edited  59305 non-null  object 
 4   item_id      59305 non-null  object 
 5   helpful      59305 non-null  object 
 6   recommend    59305 non-null  object 
 7   review       59305 non-null  object 
 8   0            0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 4.1+ MB


In [6]:
# Remove duplicate rows from the DataFrame and reset the index
df_reviews = df_reviews.drop_duplicates().reset_index(drop=True)

# Remove non-relevant columns from the DataFrame
unwanted_columns = ['funny', 'last_edited', 'helpful', 0]
df_reviews = df_reviews.drop(unwanted_columns, axis=1)

In [7]:
# Get the records with at least one null value in any column
registros_con_nulos = df_reviews[df_reviews.isnull().any(axis=1)]

print(registros_con_nulos)

                 user_id posted item_id recommend review
137                gdxsd    NaN     NaN       NaN    NaN
177    76561198094224872    NaN     NaN       NaN    NaN
2558   76561198021575394    NaN     NaN       NaN    NaN
9956             cmuir37    NaN     NaN       NaN    NaN
13528          Jaysteeny    NaN     NaN       NaN    NaN
15212             ML8989    NaN     NaN       NaN    NaN
18792  76561198079215291    NaN     NaN       NaN    NaN
19784  76561198079342142    NaN     NaN       NaN    NaN
24522  76561198061996985    NaN     NaN       NaN    NaN
25716  76561198108286351    NaN     NaN       NaN    NaN
26121     CallTripleZero    NaN     NaN       NaN    NaN
27852       Priceless612    NaN     NaN       NaN    NaN
27860      diabolical666    NaN     NaN       NaN    NaN
28377         DrScottnik    NaN     NaN       NaN    NaN
31304  76561198110176420    NaN     NaN       NaN    NaN
32151         jojoPL1987    NaN     NaN       NaN    NaN
33107        Kylee_Kylie    NaN

In [8]:
# Remove records with nulls in all specified columns
df_reviews = df_reviews.dropna(subset=['posted','item_id', 'recommend', 'review'], how='all').reset_index(drop=True)

In [9]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58431 non-null  object
 1   posted     58431 non-null  object
 2   item_id    58431 non-null  object
 3   recommend  58431 non-null  object
 4   review     58431 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [10]:
# Convert the 'posted' column to a string data type
df_reviews['posted'] = df_reviews['posted'].astype(str)

# Convert column 'item_id' to integer data type
df_reviews['item_id'] = df_reviews['item_id'].astype(int)

# Convert the 'recommend' column to a boolean data type
df_reviews['recommend'] = df_reviews['recommend'].astype(bool)

In [11]:
import re

# Function to extract year from the 'posted' column
def extract_year(date_str):
    match = re.search(r'\b\d{4}\b', date_str)
    if match:
        return int(match.group())
    else:
        return None

In [12]:
# Apply the function and create a new column 'year_posted'
df_reviews['year_posted'] = df_reviews['posted'].apply(extract_year)

# Calculate the average of the year by 'item_id'
mean_year_by_item_id = round(df_reviews.groupby('item_id')['year_posted'].mean())

# Fill missing values ​​in 'year_posted' with the corresponding mean by 'item_id
mean_year_by_item_id = df_reviews.groupby('item_id')['year_posted'].mean()
df_reviews['year_posted'] = df_reviews['year_posted'].fillna(df_reviews['item_id'].map(mean_year_by_item_id))

# Fill remaining missing values ​​in 'year_posted' with the mean corresponding to 'year_posted'
year_posted_mean = round(df_reviews['year_posted'].mean())
df_reviews['year_posted'] = df_reviews['year_posted'].fillna(year_posted_mean)

# Convert values ​​to integers for the 'year_posted' column
df_reviews['year_posted'] = df_reviews['year_posted'].astype(int)

# Delete original 'posted' column
df_reviews = df_reviews.drop('posted', axis=1)

df_reviews.head()

,user_id,item_id,recommend,review,year_posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013


sentiment_analysis

In [14]:
from textblob import TextBlob

# Define a function to perform sentiment analysis, returning a score of 0 for negative, 1 for neutral, and 2 for positive.
def analyze_sentiment(text):
    # Create a TextBlob object from the text
    analysis = TextBlob(text)

    # Assign a value based on the polarity of the analysis.
    if analysis.sentiment.polarity < 0:
        return 0
    elif analysis.sentiment.polarity == 0:
        return 1
    else:
        return 2

# Apply the analyze_sentiment() function to each row in the review column, creating a new column called sentiment_analysis.
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

# Drop the review column, since the sentiment information is now stored in the sentiment_analysis column.
df_reviews = df_reviews.drop(['review'], axis=1).reset_index(drop=True)


In [15]:
df_reviews

,user_id,item_id,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0
...,...,...,...,...,...
58426,76561198312638244,70,True,2013,2
58427,76561198312638244,362890,True,2015,2
58428,LydiaMorley,273110,True,2014,2
58429,LydiaMorley,730,True,2014,2


In [16]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   item_id             58431 non-null  int32 
 2   recommend           58431 non-null  bool  
 3   year_posted         58431 non-null  int32 
 4   sentiment_analysis  58431 non-null  int64 
dtypes: bool(1), int32(2), int64(1), object(1)
memory usage: 1.4+ MB


In [17]:
# Export procesed df_reviews
df_reviews.to_parquet('datasets/processed/reviews.parquet', index=False)


# User_items

In [18]:
# Define the file path to the compressed JSON file
file_path = os.path.join("datasets", "raw", "users_items.json.gz")

# Initialize an empty list to store the rows parsed from the file
items_row = []

# Open the file in read text mode with UTF-8 encoding and gzip decompression
with gzip.open(file_path, 'rt', encoding='utf-8') as items_file:
    # Iterate over each line in the file
    for line in items_file.readlines():
        # Convert each line to a dictionary using the ast.literal_eval function
        items_row.append(ast.literal_eval(line))

# Create a Pandas DataFrame from the processed rows
df_users_items = pd.DataFrame(items_row)

In [19]:
df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [20]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [21]:
# Convert values in the 'steam_id' column to integers
df_users_items['steam_id'] = df_users_items['steam_id'].astype('Int64')

In [22]:
# Select all columns except the last one
columns_to_check = df_users_items.iloc[:, :-1]

# Remove duplicate rows based on all columns except the last one
df_users_items = df_users_items[~columns_to_check.duplicated()]

# Filter and remove records where 'items_count' is '0' (users who don't own any games)
df_users_items = df_users_items[df_users_items['items_count'] != 0].reset_index(drop=True)

# Convert each list of dictionaries in 'items' to a DataFrame and concatenate them
df_items = pd.concat([pd.DataFrame(lst) for lst in df_users_items['items']], ignore_index=True, axis=0)

# Repeat each value of 'user_id' according to the value of 'items_count'
user_id_replicated = df_users_items['user_id'].repeat(df_users_items['items_count'])

# Create a new DataFrame with the replicated 'user_id' column
df_replicated = pd.DataFrame({'user_id_replicado': user_id_replicated}).reset_index(drop=True)

# Add the 'user_id_replicated' column as the first column of 'df_items'
df_items.insert(0, 'user_id', df_replicated['user_id_replicado'])

# Convert values in the 'item_id' column to integers
df_items['item_id'] = df_items['item_id'].astype(int)

# Remove unnecessary columns
unwanted_columns = ['playtime_2weeks']
df_items = df_items.drop(unwanted_columns, axis=1)

df_users = df_users_items.drop('items', axis=1)

In [23]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70912 entries, 0 to 70911
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      70912 non-null  object
 1   items_count  70912 non-null  int64 
 2   steam_id     70912 non-null  Int64 
 3   user_url     70912 non-null  object
dtypes: Int64(1), int64(1), object(2)
memory usage: 2.2+ MB


In [24]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int32 
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 136.0+ MB


In [25]:
# Export users and items datasets
df_users.to_parquet('datasets/processed/users.parquet', index=False)

df_items.to_parquet('datasets/processed/items.parquet', index=False)

# Games

In [ ]:
# Define the filepath for the compressed games data
file_path = os.path.join("datasets", "steam_games.json.gz")

# Initialize an empty list to store the parsed game data
games_row = []

# Open the compressed JSON file with gzip decompression and UTF-8 encoding
with gzip.open(file_path, 'rt', encoding='utf-8') as games_file:
    # Read each line of the file
    for line in games_file.readlines():
        # Convert each line (JSON string) to a Python dictionary using 'json.loads'
        data = json.loads(line)
        # Append the processed dictionary to the games_row list
        games_row.append(data)

# Create a Pandas DataFrame from the processed game data
df_games = pd.DataFrame(games_row)

In [ ]:
# Drop rows with all missing values. This removes rows where all data points are NaN.
df_games.dropna(how='all', inplace=True)

# Reset the DataFrame's index to start from 0 after dropping rows.
df_games = df_games.reset_index(drop=True)

In [ ]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [ ]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   tags          31972 non-null  object
 6   reviews_url   32133 non-null  object
 7   id            32133 non-null  object
 8   developer     28836 non-null  object
 9   release_year  29894 non-null  Int64 
dtypes: Int64(1), object(9)
memory usage: 2.5+ MB


In [ ]:
# Convert 'release_date' to datetime format in the df_games DataFrame
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Extract the year and create a new column 'release_year'
df_games['release_year'] = round(df_games['release_date'].dt.year)  # Round the year values
df_games['release_year'] = df_games['release_year'].astype('Int64')  # Convert to Int64

# Drop the original 'release_date' column
df_games = df_games.drop('release_date', axis=1)

In [ ]:
# Removal of irrelevant columns
unwanted_columns = ['specs','early_access','price']
df_games = df_games.drop(unwanted_columns, axis=1)

In [ ]:
# This class provides methods for web scraping
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import html

class WebScraper:

    # Initialize the scraper with the target URL
    def __init__(self, url):
        self.url = url
        self.soup = self.get_soup()

    # Retrieve the HTML content and create a BeautifulSoup object
    def get_soup(self):
        try:
            response = requests.get(self.url)
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        except Exception:
            return None

    # Helper function to extract year from a string
    def _extract_year(self, date_str):
        import re
        match = re.search(r'\b\d{4}\b', date_str)
        if match:
            return int(match.group())
        else:
            return None

    # Extract the title of the game
    def title(self):
        if self.soup is not None:
            # Find the element with id 'appHubAppName'
            title_element = self.soup.find('div', {'id': 'appHubAppName'})

            # Extract and clean the text within the element
            title = title_element.text.strip() if title_element else np.nan
            return title
        else:
            return np.nan

    # Extract the genre(s) of the page
    def genre(self):
        if self.soup is not None:
            # Find the element containing genre and manufacturer information
            genres_and_manufacturer_element = self.soup.find('div', {'id': 'genresAndManufacturer'})

            if genres_and_manufacturer_element:
                # Decode HTML-encoded characters
                decoded_data_panel = html.unescape('{"flow-children":"row"}')

                # Find the element containing genres specifically
                genre_element = genres_and_manufacturer_element.find('span', {'data-panel': decoded_data_panel})

                if genre_element:
                    # Extract and clean the text within the element
                    genre_string = genre_element.text.strip()

                    # Split the string into a list of genres
                    genre_list = [genre.strip() for genre in genre_string.split(',')]
                    return genre_list

            return np.nan
        else:
            return np.nan

    # Extract the developer of the page
    def developer(self):
        if self.soup is not None:
            # Find the element with id 'developers_list'
            developer_element = self.soup.find('div', {'id': 'developers_list'})

            # Extract and clean the text within the element
            developer = developer_element.text.strip() if developer_element else np.nan
            return developer
        else:
            return np.nan

    # Extract the release year of the page
    def year(self):
        if self.soup is not None:
            # Find the element containing quick information
            glance_element = self.soup.find('div', {'class': 'glance_ctn_responsive_left'})

            if glance_element:
                # Find the element containing release date
                date_element = glance_element.find('div', {'class': 'date'})

                if date_element:
                    # Extract and clean the text within the element
                    date = date_element.text.strip()

                    # Extract the year using a helper function
                    year = self._extract_year(date)
                    return year
                else:
                    return pd.NA
            else:
                return pd.NA
        else:
            return pd.NA

In [ ]:
# Function to fill in missing values for the columns: 'title', 'developer', and 'release_year'
def complete_null(row):
    ws = WebScraper(row['url'])

    row['title'] = ws.title() if pd.isna(row['title']) or row['title'] == '' else row['title']
    row['developer'] = ws.developer() if pd.isna(row['developer']) or row['developer'] == '' else row['developer']
    row['release_year'] = ws.year() if pd.isna(row['release_year']) or row['release_year'] == '' else row['release_year']

    return row

# Function to fill in missing values for the column: 'genres'
def complete_genres(row):
    ws = WebScraper(row['url'])

    row['genres'] = ws.genre() if pd.isna(row['genres']) else row['genres']

    return row

In [ ]:
# Filter DataFrame to include only rows with missing values in at least one of the columns: 'title', 'developer', or 'release_year'
df_games_nulls = df_games[df_games[['title', 'developer', 'release_year']].isna().any(axis=1)]

df_games_nulls

,publisher,genres,app_name,title,url,tags,reviews_url,id,developer,release_year
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,773570,NaN,<NA>
10,Qucheza,"[Action, Indie, Simulation, Early Access]",Uncanny Islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,"[Early Access, Action, Indie, Simulation, Surv...",http://steamcommunity.com/app/768570/reviews/?...,768570,Qucheza,<NA>
11,NaN,NaN,Icarus Six Sixty Six,NaN,http://store.steampowered.com/app/724910/Icaru...,[Casual],http://steamcommunity.com/app/724910/reviews/?...,724910,NaN,<NA>
19,NaN,NaN,After Life VR,NaN,http://store.steampowered.com/app/772590/After...,"[Early Access, Indie, VR]",http://steamcommunity.com/app/772590/reviews/?...,772590,NaN,<NA>
20,NaN,NaN,Kitty Hawk,NaN,http://store.steampowered.com/app/640250/Kitty...,"[Early Access, Action, Adventure, Indie, Casual]",http://steamcommunity.com/app/640250/reviews/?...,640250,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...
32118,NaN,NaN,Robotpencil Presents: Exercise: Brushwork,Robotpencil Presents: Exercise: Brushwork,http://store.steampowered.com/app/775640/Robot...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/775640/reviews/?...,775640,NaN,2018
32119,NaN,NaN,Robotpencil Presents: Creative Composition,Robotpencil Presents: Creative Composition,http://store.steampowered.com/app/777930/Robot...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777930/reviews/?...,777930,NaN,2018
32120,NaN,NaN,The Gamble House,The Gamble House,http://store.steampowered.com/app/775370/The_G...,[Movie],http://steamcommunity.com/app/775370/reviews/?...,775370,NaN,2016
32121,NaN,NaN,Kalen Chock Presents: 2017 Free Tutorial,Kalen Chock Presents: 2017 Free Tutorial,http://store.steampowered.com/app/777950/Kalen...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777950/reviews/?...,777950,NaN,2018


In [ ]:
from concurrent.futures import ThreadPoolExecutor

# Function to apply `complete_null` to a batch of data
def process_batch(batch):
    return batch.apply(complete_null, axis=1)

# Specify the number of threads to use
num_threads = 20

# Divide the DataFrame into batches
batch_size = len(df_games_nulls) // num_threads
batches = [df_games_nulls.iloc[i:i + batch_size] for i in range(0, len(df_games_nulls), batch_size)]

# Process the batches in parallel using a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_batch, batches))

# Combine the processed results
df_games_tdr = pd.concat(results)

# Update the original DataFrame with the processed data
df_games.update(df_games_tdr)

In [ ]:
df_games_tdr

,genres,app_name,title,url,tags,reviews_url,price,id,developer,release_year
4,"[Action, Casual, Indie, Sports]",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,2.99,773570,CGPills,2018
10,"[Action, Indie, Simulation, Early Access]",Uncanny Islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,"[Early Access, Action, Indie, Simulation, Surv...",http://steamcommunity.com/app/768570/reviews/?...,NaN,768570,Qucheza,2018
11,[Casual],Icarus Six Sixty Six,Icarus Six Sixty Six,http://store.steampowered.com/app/724910/Icaru...,[Casual],http://steamcommunity.com/app/724910/reviews/?...,Free,724910,Titmouse,2018
19,"[Indie, Early Access]",After Life VR,After Life VR,http://store.steampowered.com/app/772590/After...,"[Early Access, Indie, VR]",http://steamcommunity.com/app/772590/reviews/?...,4.99,772590,Pablo Rueda Feraud,2018
20,"[Action, Adventure, Casual, Indie, Early Access]",Kitty Hawk,Kitty Hawk,http://store.steampowered.com/app/640250/Kitty...,"[Early Access, Action, Adventure, Indie, Casual]",http://steamcommunity.com/app/640250/reviews/?...,2.99,640250,idsibidsi games,2018
...,...,...,...,...,...,...,...,...,...,...
32118,NaN,Robotpencil Presents: Exercise: Brushwork,Robotpencil Presents: Exercise: Brushwork,http://store.steampowered.com/app/775640/Robot...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/775640/reviews/?...,3.99,775640,NaN,2018.0
32119,NaN,Robotpencil Presents: Creative Composition,Robotpencil Presents: Creative Composition,http://store.steampowered.com/app/777930/Robot...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777930/reviews/?...,3.99,777930,NaN,2018.0
32120,NaN,The Gamble House,The Gamble House,http://store.steampowered.com/app/775370/The_G...,[Movie],http://steamcommunity.com/app/775370/reviews/?...,4.99,775370,NaN,2016.0
32121,NaN,Kalen Chock Presents: 2017 Free Tutorial,Kalen Chock Presents: 2017 Free Tutorial,http://store.steampowered.com/app/777950/Kalen...,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777950/reviews/?...,Free,777950,NaN,2018.0


In [ ]:
# Filter DataFrame to include only rows with missing values in the 'genres' column
df_games_genres_nulls = df_games[df_games[['genres']].isna().any(axis=1)]

df_games_genres_nulls.head()

,publisher,genres,app_name,title,url,tags,reviews_url,id,developer,release_year
4,NaN,NaN,Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,773570,CGPills,2018
11,NaN,NaN,Icarus Six Sixty Six,Icarus Six Sixty Six,http://store.steampowered.com/app/724910/Icaru...,[Casual],http://steamcommunity.com/app/724910/reviews/?...,724910,Titmouse,2018
19,NaN,NaN,After Life VR,After Life VR,http://store.steampowered.com/app/772590/After...,"[Early Access, Indie, VR]",http://steamcommunity.com/app/772590/reviews/?...,772590,Pablo Rueda Feraud,2018
20,NaN,NaN,Kitty Hawk,Kitty Hawk,http://store.steampowered.com/app/640250/Kitty...,"[Early Access, Action, Adventure, Indie, Casual]",http://steamcommunity.com/app/640250/reviews/?...,640250,idsibidsi games,2018
22,NaN,NaN,Mortars VR,Mortars VR,http://store.steampowered.com/app/711440/Morta...,"[Early Access, Strategy, Action, Indie, Casual...",http://steamcommunity.com/app/711440/reviews/?...,711440,Growl Interactive,2018


In [ ]:
# Function to apply `complete_genres` to a batch of data
def process_g_batch(batch):
    return batch.apply(complete_genres, axis=1)

# Define the number of threads to use
num_threads = 20

# Divide the DataFrame into smaller batches
batch_size = len(df_games_genres_nulls) // num_threads
batches = [df_games_genres_nulls.iloc[i:i + batch_size] for i in range(0, len(df_games_genres_nulls), batch_size)]

# Process the batches in parallel using a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_g_batch, batches))

# Merge the processed results
df_games_genres = pd.concat(results)

# Update the original DataFrame with the processed genre information
df_games.update(df_games_genres)

In [ ]:
# Impute missing values in 'developer' with values from 'publisher'
df_games['developer'] = df_games['developer'].fillna(df_games['publisher'])

# Impute missing values in 'title' with values from 'app_name'
df_games['title'] = df_games['title'].fillna(df_games['app_name'])

# Impute missing values in 'genres' with values from 'tags'
df_games['genres'] = df_games['genres'].fillna(df_games['tags'])

# Remove unnecessary columns
unwanted_columns = ['publisher', 'app_name', 'tags']
df_games = df_games.drop(unwanted_columns, axis=1)

In [ ]:
# Remove rows with missing values in the 'id' column and reset the index
df_games = df_games.dropna(subset=['id']).reset_index(drop=True)

In [ ]:
# Impute missing values in 'genres' with the string 'unknown'
df_games['genres'] = df_games['genres'].fillna('unknown')

# Impute missing values in 'developer' with the string 'unknown'
df_games['developer'] = df_games['developer'].fillna('unknown')

# Calculate the mean release year and impute missing values in 'release_year' with the rounded mean
mean_release_year = df_games['release_year'].mean()
df_games['release_year'] = df_games['release_year'].fillna(mean_release_year).round().astype('Int64')


In [ ]:
# Calculate the frequency of each genre
df_exploded = df_games['genres'].explode()
genre_counts = df_exploded.value_counts()

# Calculate the percentage distribution
distribution_percentage = (genre_counts / len(df_exploded)) * 100

# Function to get the most relevant genre for each record

def get_relevant_genre(genre_list):
    # Filter NaN from the genre list
    genre_list = [genre for genre in genre_list if not pd.isna(genre)]

    if not genre_list:
        return None
    return max(genre_list, key=lambda genre: distribution_percentage.get(genre, 0))

# Apply the function to get the most relevant genre to each row
df_games['relevant_genre'] = df_games['genres'].apply(get_relevant_genre)

In [ ]:
# Normalize Data Types

# Convert 'id' column to integer
df_games['id'] = pd.to_numeric(df_games['id'], errors='coerce').astype(int)

# Convert 'release_year' values to integer
df_games['release_year'] = df_games['release_year'].astype(int)

In [ ]:
# Renaming columns
df_games = df_games.rename(columns={
    'id': 'item_id',
    'genres': 'tags',
    'relevant_genre': 'genre'})

# Reordering columns
df_games = df_games[
    ['item_id', 'title', 'genre', 'tags', 'developer', 'release_year', 'url', 'reviews_url']]

In [ ]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item_id       32133 non-null  int64 
 1   title         32133 non-null  object
 2   genre         32133 non-null  object
 3   tags          32133 non-null  object
 4   developer     32133 non-null  object
 5   release_year  32133 non-null  int64 
 6   url           32133 non-null  object
 7   reviews_url   32133 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.0+ MB


In [ ]:
# Export games dataset
df_games.to_parquet('datasets/processed/games.parquet', index=False)

Optimizacion de DataFrames para API deployment

In [ ]:
# Merge playtime and genre data
playtime_genre = df_items[['item_id', 'user_id', 'playtime_forever']].merge(
    df_games[['item_id', 'genre', 'release_year']], on='item_id', how='inner')

# Rename columns in playtime_genre DataFrame
playtime_genre = playtime_genre.rename(
    columns={'playtime_forever': 'Horas', 'release_year': 'Año'})

# Merge user reviews and game information
users_reviews = df_reviews[['item_id', 'recommend', 'year_posted', 'sentiment_analysis']].merge(
    df_games[['item_id', 'title', 'developer']], on='item_id', how='inner')

# Rename the 'year_posted' column in users_reviews DataFrame
users_reviews = users_reviews.rename(columns={'year_posted': 'Año'})


Export API datasets

In [ ]:
# Save playtime_genre DataFrame to Parquet format
playtime_genre.to_parquet('API/datasets_API/playtime_genre.parquet', index=False)

# Save users_reviews DataFrame to Parquet format
users_reviews.to_parquet('API/datasets_API/users_reviews.parquet', index=False)